# Example-Based Model

## Initialization of the Datasets


### Source Dataset

#### Storing the Tagalog Part of Speech Data Set

In [ ]:
import pandas as pd

# Read the tagalog POS dataset
il_pos_data = pd.read_json('../../src/json data/Ilokano to Tagalog/il_pos.json')

il_doc_len = len(il_pos_data)

print('Number of documents in the dataset: {}'.format(il_doc_len))

#### Storing the Ilokano Part of Speech Data Set

In [ ]:
# Read the tagalog POS dataset
tl_pos_data = pd.read_json('../../src/json data/Tagalog to Ilokano/tl_pos.json')

tl_doc_len = len(tl_pos_data)

tl_pos_data.head()

#### Storing the Tagalog Part of Speech Structure

In [ ]:
dict_sen_poss = pd.DataFrame(il_pos_data['POS'])

dict_sen_poss.columns = ['Ilokano POS']

#### Storing the Ilokano Part of Speech 

In [ ]:
dict_sen_poss['Tagalog POS'] = tl_pos_data['POS']

dict_sen_poss.head()

### Tagalog to Ilokano Dictionary

In [ ]:
tl_sen_poss_list = dict_sen_poss['Tagalog POS']
il_sen_poss_list = dict_sen_poss['Ilokano POS']
"""
putting the POS of the sentences in a list object
"""

dict_il_tl_sw = pd.DataFrame(columns=['Ilokano Single Words', 'Tagalog Single Words'])
dict_il_tl_vb = pd.DataFrame(columns=['Ilokano Verb', 'Tagalog Verb'])
dict_il_tl_nn = pd.DataFrame(columns=['Ilokano Noun', 'Tagalog Noun'])
dict_il_tl_jj = pd.DataFrame(columns=['Ilokano Adjective', 'Tagalog Adjective'])
dict_il_tl_rb = pd.DataFrame(columns=['Ilokano Adverb', 'Tagalog Adverb'])
dict_il_tl_cc = pd.DataFrame(columns=['Ilokano Conjunction', 'Tagalog Conjunction'])
dict_il_tl_pr = pd.DataFrame(columns=['Ilokano Preposition', 'Tagalog Preposition'])
dict_il_tl_dt = pd.DataFrame(columns=['Ilokano Determiner', 'Tagalog Determiner'])


## Appending in the List

### For Verb List

In [ ]:
def append_vb_list(il_verb, il_verb_list, tl_verb, tl_verb_list, curr_tl_pos, next_tl_pos, next2_tl_pos, matched, sp_index, wp_index, tl_verb_count_list, tl_verb_count, il_verb_count_list, il_verb_count, il_verb_term_count, il_verb_sen, il_verb_count_list_tf, il_sen_len, tl_verb_sen, tl_verb_count_list_idf, tl_verb_count_list_tf, tl_verb_count_tf, tl_sen_len):
    
    if il_verb not in il_verb_list: # il_verb_list = json ftle of verbs
        """
        if the verb is not in the list
        """
        il_verb_list.append(il_verb)
        il_verb_count.append(1) 
        il_verb_term_count.append(il_verb_count) # not necessary
        
        if il_verb not in il_verb_sen: # il_verb_sen - list of verbs in a single sentence
            il_verb_sen.append(il_verb)
            il_verb_count_list.append(1)
            il_verb_count_list_tf.append([1/il_sen_len])

        inDict = False
        
    else:
        """
        if the verb is in the list
        """
        temp_index = il_verb_list.index(il_verb)
        
        temp_verb_index = il_verb_list[temp_index].index(il_verb[0]) # not necessary
        il_verb_term_count[temp_index][temp_verb_index] += 1

        if il_verb not in il_verb_sen:
            il_verb_sen.append(il_verb)
            il_verb_count_list[temp_index] += 1
            il_verb_count_list_tf[temp_index].append(1/il_sen_len)
        else:
            try:
                il_verb_count_list_tf[temp_index][sp_index] += (1/il_sen_len)
            except:
                il_verb_count_list_tf[temp_index].append(1/il_sen_len)
        
        inDict = True

    """
    append the the verb in the tagalog verb
    """

    if curr_tl_pos == 'VB' and wp_index not in matched:
        """
        if VB : VB
        if the tlokano POS is a verb
        """
        temp_verb = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_verb.append(temp_verb)
        matched.append(wp_index)
        
    elif curr_tl_pos == 'DT' and next_tl_pos == 'VB' and wp_index not in matched:    
        """
        if VB : DT VB
        if the tlokano POS is a determiner and the next POS is a verb
        eg. Ntlalang : ti Aramid
        """
        temp_verb = tl_pos_data['Tokenized'][sp_index][wp_index + 1]
        tl_verb.append(temp_verb)
        matched.append(wp_index)
        matched.append(wp_index + 1) 

    elif curr_tl_pos == 'NN' and next_tl_pos == 'VB' and wp_index not in matched:    
        """
        if VB : NN VB
        if the tlokano POS is a determiner and the next POS is a verb
        """
        temp_verb = tl_pos_data['Tokenized'][sp_index][wp_index + 1]
        tl_verb.append(temp_verb)
        matched.append(wp_index + 1)
        
    elif curr_tl_pos == 'DT' and next_tl_pos == 'NN' and next2_tl_pos == 'VB' and wp_index not in matched:
        """
        if VB DT NN : DT NN VB
        if the tlokano POS is a determiner and the next POS is a verb
        """
        temp_verb = tl_pos_data['Tokenized'][sp_index][wp_index + 2]
        tl_verb.append(temp_verb)
        matched.append(wp_index + 2)
        
    else:
        """
        if VB : Other POS
        if the tlokano POS is not a verb
        """
        tl_verb.append('None')
        matched.append(wp_index)

    if not inDict:
        tl_verb_list.append(tl_verb)
        if tl_verb[0] == 'None':
            tl_verb_count.append(0)
            tl_verb_count_tf.append(round((0/tl_sen_len), 6))
        else:
            tl_verb_count.append(1)
            tl_verb_count_tf.append(round((1/tl_sen_len), 6))
        tl_verb_count_list.append(tl_verb_count)
        tl_verb_count_list_tf.append(tl_verb_count_tf)
        
        if tl_verb[0] not in tl_verb_sen:
            tl_verb_sen.append(tl_verb[0])
            if tl_verb[0] == 'None':
                tl_verb_count_list_idf.append([0])
            else:
                tl_verb_count_list_idf.append([1])
        else:
            tl_verb_count_list_idf.append([0])
            
    else:
        if tl_verb[0] not in tl_verb_list[temp_index]:
            tl_verb_list[temp_index].append(tl_verb[0]) # temp_index = row index of the verb
            if tl_verb[0] == 'None':
                tl_verb_count_list[temp_index].append(0)
                tl_verb_count_list_tf[temp_index].append(round((0/tl_sen_len), 6))
            else:
                tl_verb_count_list[temp_index].append(1)
                tl_verb_count_list_tf[temp_index].append(round((1/tl_sen_len), 6)) 
            
            if tl_verb[0] not in tl_verb_sen: # tl_verb_sen = list of verbs in  a single sentence
                tl_verb_sen.append(tl_verb[0])
                if tl_verb[0] == 'None':
                    tl_verb_count_list_idf[temp_index].append(0)
                else:
                    tl_verb_count_list_idf[temp_index].append(1)
            else:
                tl_verb_count_list_idf[temp_index].append(0)
            
        else: # if the verb is already in tl_verb_list
            temp_verb_index = tl_verb_list[temp_index].index(tl_verb[0])
            if tl_verb[0] == 'None':
                tl_verb_count_list[temp_index][temp_verb_index] += 0
                tl_verb_count_list_tf[temp_index][temp_verb_index] += (0/tl_sen_len)
            else:
                tl_verb_count_list[temp_index][temp_verb_index] += 1
                tl_verb_count_list_tf[temp_index][temp_verb_index] += (1/tl_sen_len)
            tl_verb_count_list_tf[temp_index][temp_verb_index] = round(tl_verb_count_list_tf[temp_index][temp_verb_index], 6)
            
            if tl_verb[0] not in tl_verb_sen:
                tl_verb_sen.append(tl_verb[0])
                if tl_verb[0] == 'None':
                    tl_verb_count_list_idf[temp_index][temp_verb_index] += 0
                else:
                    tl_verb_count_list_idf[temp_index][temp_verb_index] += 1

### For Noun List

In [ ]:
def append_nn_list(il_noun, il_noun_list, tl_noun, tl_noun_list, curr_tl_pos, next_tl_pos, matched, sp_index, wp_index, tl_noun_count_list, tl_noun_count, il_noun_count_list, il_noun_count, il_noun_term_count, il_noun_sen, il_noun_count_list_tf, il_sen_len, tl_noun_sen, tl_noun_count_list_idf, tl_noun_count_list_tf, tl_noun_count_tf, tl_sen_len):
    
    if il_noun not in il_noun_list:
        """
        if the verb is not in the list
        """
        il_noun_list.append(il_noun)
        il_noun_count.append(1) 
        il_noun_term_count.append(il_noun_count)
        
        if il_noun not in il_noun_sen:
            il_noun_sen.append(il_noun)
            il_noun_count_list.append(1)
            il_noun_count_list_tf.append([1/il_sen_len])

        inDict = False
        
    else:
        """
        if the verb is in the list
        """
        temp_index = il_noun_list.index(il_noun)

        temp_noun_index = il_noun_list[temp_index].index(il_noun[0])
        il_noun_term_count[temp_index][temp_noun_index] += 1
        
        if il_noun not in il_noun_sen:
            il_noun_sen.append(il_noun)
            il_noun_count_list[temp_index] += 1
            il_noun_count_list_tf[temp_index].append(1/il_sen_len)
        else:
            try:
                il_noun_count_list_tf[temp_index][sp_index] += (1/il_sen_len)
            except:
                il_noun_count_list_tf[temp_index].append(1/il_sen_len)
        
        inDict = True

    """
    append the the noun in the tagalog noun
    """

    if curr_tl_pos == 'NN':
        """
        if NN : NN
        if the tlokano POS is a noun
        """
        temp_noun = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_noun.append(temp_noun)
        matched.append(wp_index)
        
        
    elif curr_tl_pos == 'DT' and next_tl_pos == 'NN':    
        """
        if NN : DT NN
        if the tlokano POS is a determiner and the next POS is a noun
        """
        temp_next_noun = tl_pos_data['Tokenized'][sp_index][wp_index + 1]
        tl_noun.append(temp_next_noun) 
        matched.append(wp_index + 1)
          
    else:
        """
        if NN : Other POS
        if the tlokano POS is not a noun
        """
        tl_noun.append('None')
        matched.append(wp_index)

    if not inDict:
        tl_noun_list.append(tl_noun)
        if tl_noun[0] == 'None':
            tl_noun_count.append(0)
            tl_noun_count_tf.append(round((0/tl_sen_len), 6))
        else:
            tl_noun_count.append(1)
            tl_noun_count_tf.append(round((1/tl_sen_len), 6))
        tl_noun_count_list.append(tl_noun_count)
        tl_noun_count_list_tf.append(tl_noun_count_tf)
        
        if tl_noun[0] not in tl_noun_sen:
            tl_noun_sen.append(tl_noun[0])
            if tl_noun[0] == 'None':
                tl_noun_count_list_idf.append([0])
            else:
                tl_noun_count_list_idf.append([1])
        else:
            tl_noun_count_list_idf.append([0])
            
    else:
        if tl_noun[0] not in tl_noun_list[temp_index]:
            tl_noun_list[temp_index].append(tl_noun[0]) # temp_index = row index of the noun
            if tl_noun[0] == 'None':
                tl_noun_count_list[temp_index].append(0)
                tl_noun_count_list_tf[temp_index].append(round((0/tl_sen_len), 6))
            else:
                tl_noun_count_list[temp_index].append(1)
                tl_noun_count_list_tf[temp_index].append(round((1/tl_sen_len), 6)) 
            
            if tl_noun[0] not in tl_noun_sen: # tl_noun_sen = list of nouns in  a single sentence
                tl_noun_sen.append(tl_noun[0])
                if tl_noun[0] == 'None':
                    tl_noun_count_list_idf[temp_index].append(0)
                else:
                    tl_noun_count_list_idf[temp_index].append(1)
            else:
                tl_noun_count_list_idf[temp_index].append(0)
            
        else: # if the noun is already in tl_noun_list
            temp_noun_index = tl_noun_list[temp_index].index(tl_noun[0])
            if tl_noun[0] == 'None':
                tl_noun_count_list[temp_index][temp_noun_index] += 0
                tl_noun_count_list_tf[temp_index][temp_noun_index] += (0/tl_sen_len)
            else:
                tl_noun_count_list[temp_index][temp_noun_index] += 1
                tl_noun_count_list_tf[temp_index][temp_noun_index] += (1/tl_sen_len)
            tl_noun_count_list_tf[temp_index][temp_noun_index] = round(tl_noun_count_list_tf[temp_index][temp_noun_index], 6)
            
            if tl_noun[0] not in tl_noun_sen:
                tl_noun_sen.append(tl_noun[0])
                if tl_noun[0] == 'None':
                    tl_noun_count_list_idf[temp_index][temp_noun_index] += 0
                else:
                    tl_noun_count_list_idf[temp_index][temp_noun_index] += 1

### For Adjective List

In [ ]:
def append_jj_list(il_adj, il_adj_list, tl_adj, tl_adj_list, curr_tl_pos, next_tl_pos, next2_tl_pos, matched, sp_index, wp_index, tl_adj_count_list, tl_adj_count, il_adj_count_list, il_adj_count, il_adj_term_count, il_adj_sen, il_adj_count_list_tf, il_sen_len, tl_adj_sen, tl_adj_count_list_idf, tl_adj_count_list_tf, tl_adj_count_tf, tl_sen_len):
    
    if il_adj not in il_adj_list:
        """
        if the verb is not in the list
        """
        il_adj_list.append(il_adj)
        il_adj_count.append(1) 
        il_adj_term_count.append(il_adj_count)
        
        if il_adj not in il_adj_sen:
            il_adj_sen.append(il_adj)
            il_adj_count_list.append(1)
            il_adj_count_list_tf.append([1/il_sen_len])

        inDict = False
        
    else:
        """
        if the verb is in the list
        """
        temp_index = il_adj_list.index(il_adj)

        temp_adj_index = il_adj_list[temp_index].index(il_adj[0])
        il_adj_term_count[temp_index][temp_adj_index] += 1
        
        if il_adj not in il_adj_sen:
            il_adj_sen.append(il_adj)
            il_adj_count_list[temp_index] += 1
            il_adj_count_list_tf[temp_index].append(1/il_sen_len)
        else:
            try:
                il_adj_count_list_tf[temp_index][sp_index] += (1/il_sen_len)
            except:
                il_adj_count_list_tf[temp_index].append(1/il_sen_len)
        
        inDict = True

    """
    append the the adj in the tagalog adj
    """

    if curr_tl_pos == 'JJ':
        """
        if JJ : JJ
        if the tlokano POS is an adj
        """
        temp_adj = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_adj.append(temp_adj)
        matched.append(wp_index)
        
    elif curr_tl_pos == 'DT' and next_tl_pos == 'JJ':    
        """
        if JJ : DT JJ
        if the tlokano POS is a determiner and the next POS is an adj
        eg. mabubuting : ken naimbag
        """
        temp_next_adj = tl_pos_data['Tokenized'][sp_index][wp_index + 1]
        tl_adj.append(temp_next_adj) 
        matched.append(wp_index + 1)
        
    elif curr_tl_pos == 'JJ' and next_tl_pos == 'DT' and next2_tl_pos == 'NN':    
        """
        if JJ : JJ NN
        if the tlokano POS is a determiner and the next POS is an adj
        eg. mabubuting : ken naimbag
        """
        temp_curr_adj = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_adj.append(temp_curr_adj) 
        matched.append(wp_index + 1)

    else:
        """
        if JJ : Other POS
        if the tlokano POS is not an adj
        """
        tl_adj.append('None')
        matched.append(wp_index)

    if not inDict:
        tl_adj_list.append(tl_adj)
        if tl_adj[0] == 'None':
            tl_adj_count.append(0)
            tl_adj_count_tf.append(round((0/tl_sen_len), 6))
        else:
            tl_adj_count.append(1)
            tl_adj_count_tf.append(round((1/tl_sen_len), 6))
        tl_adj_count_list.append(tl_adj_count)
        tl_adj_count_list_tf.append(tl_adj_count_tf)
        
        if tl_adj[0] not in tl_adj_sen:
            tl_adj_sen.append(tl_adj[0])
            if tl_adj[0] == 'None':
                tl_adj_count_list_idf.append([0])
            else:
                tl_adj_count_list_idf.append([1])
        else:
            tl_adj_count_list_idf.append([0])
            
    else:
        if tl_adj[0] not in tl_adj_list[temp_index]:
            tl_adj_list[temp_index].append(tl_adj[0]) # temp_index = row index of the adj
            if tl_adj[0] == 'None':
                tl_adj_count_list[temp_index].append(0)
                tl_adj_count_list_tf[temp_index].append(round((0/tl_sen_len), 6))
            else:
                tl_adj_count_list[temp_index].append(1)
                tl_adj_count_list_tf[temp_index].append(round((1/tl_sen_len), 6)) 
            
            if tl_adj[0] not in tl_adj_sen: # tl_adj_sen = list of adjs in  a single sentence
                tl_adj_sen.append(tl_adj[0])
                if tl_adj[0] == 'None':
                    tl_adj_count_list_idf[temp_index].append(0)
                else:
                    tl_adj_count_list_idf[temp_index].append(1)
            else:
                tl_adj_count_list_idf[temp_index].append(0)
            
        else: # if the adj is already in tl_adj_list
            temp_adj_index = tl_adj_list[temp_index].index(tl_adj[0])
            if tl_adj[0] == 'None':
                tl_adj_count_list[temp_index][temp_adj_index] += 0
                tl_adj_count_list_tf[temp_index][temp_adj_index] += (0/tl_sen_len)
            else:
                tl_adj_count_list[temp_index][temp_adj_index] += 1
                tl_adj_count_list_tf[temp_index][temp_adj_index] += (1/tl_sen_len)
            tl_adj_count_list_tf[temp_index][temp_adj_index] = round(tl_adj_count_list_tf[temp_index][temp_adj_index], 6)
            
            if tl_adj[0] not in tl_adj_sen:
                tl_adj_sen.append(tl_adj[0])
                if tl_adj[0] == 'None':
                    tl_adj_count_list_idf[temp_index][temp_adj_index] += 0
                else:
                    tl_adj_count_list_idf[temp_index][temp_adj_index] += 1

### For Adverb List


In [ ]:
def append_rb_list(il_adv, il_adv_list, tl_adv, tl_adv_list, curr_tl_pos, next_tl_pos, next2_tl_pos, next3_tl_pos, prev_tl_pos, matched, sp_index, wp_index, tl_adv_count_list, tl_adv_count, il_adv_count_list, il_adv_count, il_adv_term_count, il_adv_sen, il_adv_count_list_tf, il_sen_len, tl_adv_sen, tl_adv_count_list_idf, tl_adv_count_list_tf, tl_adv_count_tf, tl_sen_len):
    
    if il_adv not in il_adv_list:
        """
        if the verb is not in the list
        """
        il_adv_list.append(il_adv)
        il_adv_count.append(1) 
        il_adv_term_count.append(il_adv_count)
        
        if il_adv not in il_adv_sen:
            il_adv_sen.append(il_adv)
            il_adv_count_list.append(1)
            il_adv_count_list_tf.append([1/il_sen_len])

        inDict = False
        
    else:
        """
        if the verb is in the list
        """
        temp_index = il_adv_list.index(il_adv)

        temp_adv_index = il_adv_list[temp_index].index(il_adv[0])
        il_adv_term_count[temp_index][temp_adv_index] += 1
        
        if il_adv not in il_adv_sen:
            il_adv_sen.append(il_adv)
            il_adv_count_list[temp_index] += 1
            il_adv_count_list_tf[temp_index].append(1/il_sen_len)
        else:
            try:
                il_adv_count_list_tf[temp_index][sp_index] += (1/il_sen_len)
            except:
                il_adv_count_list_tf[temp_index].append(1/il_sen_len)
        
        inDict = True

    """
    append the the verb in the tagalog verb
    """

    if curr_tl_pos == 'RB':
        """
        if RB : RB
        if the tlokano POS is a adverb
        """
        temp_adverb = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_adv.append(temp_adverb)
        matched.append(wp_index)
        
    elif curr_tl_pos == 'DT' and next_tl_pos == 'RB':    
        """
        if RB : DT RB
        """
        temp_next_adverb = tl_pos_data['Tokenized'][sp_index][wp_index + 1]
        tl_adv.append(temp_next_adverb) 
        matched.append(wp_index + 1)
  
        
    elif curr_tl_pos == 'DT' and next_tl_pos == 'NN' and next2_tl_pos == 'DT' and next3_tl_pos == 'RB' :  
        """
        if RB : DT NN DT RB
        
        """
        temp_adverb = tl_pos_data['Tokenized'][sp_index][wp_index + 3]
        tl_adv.append(temp_adverb)
        matched.append(wp_index + 3)
        
    elif curr_tl_pos == 'DT' and prev_tl_pos == 'RB':
        """
        if RB : DT with RB behind DT
        
        """
        temp_adverb = tl_pos_data['Tokenized'][sp_index][wp_index - 1]
        tl_adv.append(temp_adverb)
        matched.append(wp_index - 1)
        
    else:
        """
        if RB : Other POS
        if the tlokano POS is not a adverb
        """
        tl_adv.append('None')
        matched.append(wp_index)

    if not inDict:
        tl_adv_list.append(tl_adv)
        if tl_adv[0] == 'None':
            tl_adv_count.append(0)
            tl_adv_count_tf.append(round((0/tl_sen_len), 6))
        else:
            tl_adv_count.append(1)
            tl_adv_count_tf.append(round((1/tl_sen_len), 6))
        tl_adv_count_list.append(tl_adv_count)
        tl_adv_count_list_tf.append(tl_adv_count_tf)
        
        if tl_adv[0] not in tl_adv_sen:
            tl_adv_sen.append(tl_adv[0])
            if tl_adv[0] == 'None':
                tl_adv_count_list_idf.append([0])
            else:
                tl_adv_count_list_idf.append([1])
        else:
            tl_adv_count_list_idf.append([0])
            
    else:
        if tl_adv[0] not in tl_adv_list[temp_index]:
            tl_adv_list[temp_index].append(tl_adv[0]) # temp_index = row index of the adv
            if tl_adv[0] == 'None':
                tl_adv_count_list[temp_index].append(0)
                tl_adv_count_list_tf[temp_index].append(round((0/tl_sen_len), 6))
            else:
                tl_adv_count_list[temp_index].append(1)
                tl_adv_count_list_tf[temp_index].append(round((1/tl_sen_len), 6)) 
            
            if tl_adv[0] not in tl_adv_sen: # tl_adv_sen = list of advs in  a single sentence
                tl_adv_sen.append(tl_adv[0])
                if tl_adv[0] == 'None':
                    tl_adv_count_list_idf[temp_index].append(0)
                else:
                    tl_adv_count_list_idf[temp_index].append(1)
            else:
                tl_adv_count_list_idf[temp_index].append(0)
            
        else: # if the adv is already in tl_adv_list
            temp_adv_index = tl_adv_list[temp_index].index(tl_adv[0])
            if tl_adv[0] == 'None':
                tl_adv_count_list[temp_index][temp_adv_index] += 0
                tl_adv_count_list_tf[temp_index][temp_adv_index] += (0/tl_sen_len)
            else:
                tl_adv_count_list[temp_index][temp_adv_index] += 1
                tl_adv_count_list_tf[temp_index][temp_adv_index] += (1/tl_sen_len)
            tl_adv_count_list_tf[temp_index][temp_adv_index] = round(tl_adv_count_list_tf[temp_index][temp_adv_index], 6)
            
            if tl_adv[0] not in tl_adv_sen:
                tl_adv_sen.append(tl_adv[0])
                if tl_adv[0] == 'None':
                    tl_adv_count_list_idf[temp_index][temp_adv_index] += 0
                else:
                    tl_adv_count_list_idf[temp_index][temp_adv_index] += 1

### For Conjunction List

In [ ]:
def append_cc_list(il_conj, il_conj_list, tl_conj, tl_conj_list, curr_tl_pos, matched, sp_index, wp_index, tl_conj_count_list, tl_conj_count, il_conj_count_list, il_conj_count, il_conj_term_count, il_conj_sen, il_conj_count_list_tf, il_sen_len, tl_conj_sen, tl_conj_count_list_idf, tl_conj_count_list_tf, tl_conj_count_tf, tl_sen_len):
    
    if il_conj not in il_conj_list: # il_conj_list = json ftle of verbs
        """
        if the verb is not in the list
        """
        il_conj_list.append(il_conj)
        il_conj_count.append(1) 
        il_conj_term_count.append(il_conj_count)
        
        if il_conj not in il_conj_sen:
            il_conj_sen.append(il_conj)
            il_conj_count_list.append(1)
            il_conj_count_list_tf.append([1/il_sen_len])

        inDict = False
        
    else:
        """
        if the verb is in the list
        """
        temp_index = il_conj_list.index(il_conj)

        temp_conj_index = il_conj_list[temp_index].index(il_conj[0])
        il_conj_term_count[temp_index][temp_conj_index] += 1
        
        if il_conj not in il_conj_sen:
            il_conj_sen.append(il_conj)
            il_conj_count_list[temp_index] += 1
            il_conj_count_list_tf[temp_index].append(1/il_sen_len)
        else:
            try:
                il_conj_count_list_tf[temp_index][sp_index] += (1/il_sen_len)
            except:
                il_conj_count_list_tf[temp_index].append(1/il_sen_len)
        
        inDict = True

    """
    append the the conj in the tagalog conj
    """

    if curr_tl_pos == 'CC':
        """
        if CC : CC
        if the tlokano POS is a conj
        """
        temp_conj = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_conj.append(temp_conj)
        matched.append(wp_index)
          
    else:
        """
        if CC : Other POS
        if the tlokano POS is not a conj
        """
        tl_conj.append('None')
        matched.append(wp_index)

    if not inDict:
        tl_conj_list.append(tl_conj)
        if tl_conj[0] == 'None':
            tl_conj_count.append(0)
            tl_conj_count_tf.append(round((0/tl_sen_len), 6))
        else:
            tl_conj_count.append(1)
            tl_conj_count_tf.append(round((1/tl_sen_len), 6))
        tl_conj_count_list.append(tl_conj_count)
        tl_conj_count_list_tf.append(tl_conj_count_tf)
        
        if tl_conj[0] not in tl_conj_sen:
            tl_conj_sen.append(tl_conj[0])
            if tl_conj[0] == 'None':
                tl_conj_count_list_idf.append([0])
            else:
                tl_conj_count_list_idf.append([1])
        else:
            tl_conj_count_list_idf.append([0])
            
    else:
        if tl_conj[0] not in tl_conj_list[temp_index]:
            tl_conj_list[temp_index].append(tl_conj[0]) # temp_index = row index of the conj
            if tl_conj[0] == 'None':
                tl_conj_count_list[temp_index].append(0)
                tl_conj_count_list_tf[temp_index].append(round((0/tl_sen_len), 6))
            else:
                tl_conj_count_list[temp_index].append(1)
                tl_conj_count_list_tf[temp_index].append(round((1/tl_sen_len), 6)) 
            
            if tl_conj[0] not in tl_conj_sen: # tl_conj_sen = list of conjs in  a single sentence
                tl_conj_sen.append(tl_conj[0])
                if tl_conj[0] == 'None':
                    tl_conj_count_list_idf[temp_index].append(0)
                else:
                    tl_conj_count_list_idf[temp_index].append(1)
            else:
                tl_conj_count_list_idf[temp_index].append(0)
            
        else: # if the conj is already in tl_conj_list
            temp_conj_index = tl_conj_list[temp_index].index(tl_conj[0])
            if tl_conj[0] == 'None':
                tl_conj_count_list[temp_index][temp_conj_index] += 0
                tl_conj_count_list_tf[temp_index][temp_conj_index] += (0/tl_sen_len)
            else:
                tl_conj_count_list[temp_index][temp_conj_index] += 1
                tl_conj_count_list_tf[temp_index][temp_conj_index] += (1/tl_sen_len)
            tl_conj_count_list_tf[temp_index][temp_conj_index] = round(tl_conj_count_list_tf[temp_index][temp_conj_index], 6)
            
            if tl_conj[0] not in tl_conj_sen:
                tl_conj_sen.append(tl_conj[0])
                if tl_conj[0] == 'None':
                    tl_conj_count_list_idf[temp_index][temp_conj_index] += 0
                else:
                    tl_conj_count_list_idf[temp_index][temp_conj_index] += 1

### For Preposition List

In [ ]:
def append_pr_list(il_prepo, il_prepo_list, tl_prepo, tl_prepo_list, curr_tl_pos, next_tl_pos, next2_tl_pos, matched, sp_index, wp_index, tl_prepo_count_list, tl_prepo_count, il_prepo_count_list, il_prepo_count, il_prepo_term_count, il_prepo_sen, il_prepo_count_list_tf, il_sen_len, tl_prepo_sen, tl_prepo_count_list_idf, tl_prepo_count_list_tf, tl_prepo_count_tf, tl_sen_len):
    
    if il_prepo not in il_prepo_list: # il_prepo_list = json ftle of verbs
        """
        if the verb is not in the list
        """
        il_prepo_list.append(il_prepo)
        il_prepo_count.append(1) 
        il_prepo_term_count.append(il_prepo_count)
        
        if il_prepo not in il_prepo_sen:
            il_prepo_sen.append(il_prepo)
            il_prepo_count_list.append(1)
            il_prepo_count_list_tf.append([1/il_sen_len])

        inDict = False
        
    else:
        """
        if the verb is in the list
        """
        temp_index = il_prepo_list.index(il_prepo)

        temp_prepo_index = il_prepo_list[temp_index].index(il_prepo[0])
        il_prepo_term_count[temp_index][temp_prepo_index] += 1
        
        if il_prepo not in il_prepo_sen:
            il_prepo_sen.append(il_prepo)
            il_prepo_count_list[temp_index] += 1
            il_prepo_count_list_tf[temp_index].append(1/il_sen_len)
        else:
            try:
                il_prepo_count_list_tf[temp_index][sp_index] += (1/il_sen_len)
            except:
                il_prepo_count_list_tf[temp_index].append(1/il_sen_len)
        
        inDict = True
    
    """
    append the the verb in the tagalog verb
    """
    
    if curr_tl_pos == 'PR' and wp_index not in matched:
        """
        if PR : PR
        if the tlokano POS is a verb
        """
        temp_prepo = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_prepo.append(temp_prepo)
        matched.append(wp_index)
    if curr_tl_pos == 'DT' and next_tl_pos == 'PR' and wp_index not in matched:
        """
        if PR : PR
        if the tlokano POS is a verb
        """
        temp_prepo = tl_pos_data['Tokenized'][sp_index][wp_index + 1]
        tl_prepo.append(temp_prepo)
        matched.append(wp_index + 1)
    if curr_tl_pos == 'DT' and next_tl_pos == 'PR' and next2_tl_pos == 'DT' and wp_index not in matched:
        """
        if PR : DT PR DT
        if the tlokano prepo is sandwiched between 2 determiners
        """
        temp_prepo = tl_pos_data['Tokenized'][sp_index][wp_index + 1]
        tl_prepo.append(temp_prepo)
        matched.append(wp_index + 1)
    if curr_tl_pos == 'PR' and next_tl_pos == 'DT' and next2_tl_pos == 'NN' and wp_index not in matched:
        """
        if PR : DT PR DT
        if the tlokano prepo is sandwiched between 2 determiners
        """
        temp_prepo = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_prepo.append(temp_prepo)
        matched.append(wp_index)
    
    else:
        """
        if VB : Other POS
        if the tlokano POS is not a verb
        """
        tl_prepo.append('None')
        matched.append(wp_index)
    
    if not inDict:
        tl_prepo_list.append(tl_prepo)
        if tl_prepo[0] == 'None':
            tl_prepo_count.append(0)
            tl_prepo_count_tf.append(round((0/tl_sen_len), 6))
        else:
            tl_prepo_count.append(1)
            tl_prepo_count_tf.append(round((1/tl_sen_len), 6))
        tl_prepo_count_list.append(tl_prepo_count)
        tl_prepo_count_list_tf.append(tl_prepo_count_tf)
        
        if tl_prepo[0] not in tl_prepo_sen:
            tl_prepo_sen.append(tl_prepo[0])
            if tl_prepo[0] == 'None':
                tl_prepo_count_list_idf.append([0])
            else:
                tl_prepo_count_list_idf.append([1])
        else:
            tl_prepo_count_list_idf.append([0])
            
    else:
        if tl_prepo[0] not in tl_prepo_list[temp_index]:
            tl_prepo_list[temp_index].append(tl_prepo[0]) # temp_index = row index of the prepo
            if tl_prepo[0] == 'None':
                tl_prepo_count_list[temp_index].append(0)
                tl_prepo_count_list_tf[temp_index].append(round((0/tl_sen_len), 6))
            else:
                tl_prepo_count_list[temp_index].append(1)
                tl_prepo_count_list_tf[temp_index].append(round((1/tl_sen_len), 6)) 
            
            if tl_prepo[0] not in tl_prepo_sen: # tl_prepo_sen = list of prepos in  a single sentence
                tl_prepo_sen.append(tl_prepo[0])
                if tl_prepo[0] == 'None':
                    tl_prepo_count_list_idf[temp_index].append(0)
                else:
                    tl_prepo_count_list_idf[temp_index].append(1)
            else:
                tl_prepo_count_list_idf[temp_index].append(0)
            
        else: # if the prepo is already in tl_prepo_list
            temp_prepo_index = tl_prepo_list[temp_index].index(tl_prepo[0])
            if tl_prepo[0] == 'None':
                tl_prepo_count_list[temp_index][temp_prepo_index] += 0
                tl_prepo_count_list_tf[temp_index][temp_prepo_index] += (0/tl_sen_len)
            else:
                tl_prepo_count_list[temp_index][temp_prepo_index] += 1
                tl_prepo_count_list_tf[temp_index][temp_prepo_index] += (1/tl_sen_len)
            tl_prepo_count_list_tf[temp_index][temp_prepo_index] = round(tl_prepo_count_list_tf[temp_index][temp_prepo_index], 6)
            
            if tl_prepo[0] not in tl_prepo_sen:
                tl_prepo_sen.append(tl_prepo[0])
                if tl_prepo[0] == 'None':
                    tl_prepo_count_list_idf[temp_index][temp_prepo_index] += 0
                else:
                    tl_prepo_count_list_idf[temp_index][temp_prepo_index] += 1

### For Determiner List

In [ ]:
def append_dt_list(il_dt, il_dt_list, tl_dt, tl_dt_list, curr_tl_pos, next_tl_pos, next2_tl_pos, matched, sp_index, wp_index, tl_dt_count_list, tl_dt_count, il_dt_count_list, il_dt_count, il_dt_term_count, il_dt_sen, il_dt_count_list_tf, il_sen_len, tl_dt_sen, tl_dt_count_list_idf, tl_dt_count_list_tf, tl_dt_count_tf, tl_sen_len):
    
    if il_dt not in il_dt_list: # il_dt_list = json ftle of verbs
        """
        if the verb is not in the list
        """
        il_dt_list.append(il_dt)
        il_dt_count.append(1) 
        il_dt_term_count.append(il_dt_count)
        
        if il_dt not in il_dt_sen:
            il_dt_sen.append(il_dt)
            il_dt_count_list.append(1)
            il_dt_count_list_tf.append([1/il_sen_len])

        inDict = False
        
    else:
        """
        if the verb is in the list
        """
        temp_index = il_dt_list.index(il_dt)

        temp_dt_index = il_dt_list[temp_index].index(il_dt[0])
        il_dt_term_count[temp_index][temp_dt_index] += 1
        
        if il_dt not in il_dt_sen:
            il_dt_sen.append(il_dt)
            il_dt_count_list[temp_index] += 1
            il_dt_count_list_tf[temp_index].append(1/il_sen_len)
        else:
            try:
                il_dt_count_list_tf[temp_index][sp_index] += (1/il_sen_len)
            except:
                il_dt_count_list_tf[temp_index].append(1/il_sen_len)
        
        inDict = True
    
    """
    append the the verb in the tagalog verb
    """
    
    if curr_tl_pos == 'DT' and wp_index not in matched:
        """
        if PR : PR
        if the tlokano POS is a verb
        """
        temp_prepo = tl_pos_data['Tokenized'][sp_index][wp_index]
        tl_dt.append(temp_prepo)
        matched.append(wp_index)
    
    else:
        """
        if VB : Other POS
        if the tlokano POS is not a verb
        """
        tl_dt.append('None')
        matched.append(wp_index)
        
    if not inDict:
        tl_dt_list.append(tl_dt)
        if tl_dt[0] == 'None':
            tl_dt_count.append(0)
            tl_dt_count_tf.append(round((0/tl_sen_len), 6))
        else:
            tl_dt_count.append(1)
            tl_dt_count_tf.append(round((1/tl_sen_len), 6))
        tl_dt_count_list.append(tl_dt_count)
        tl_dt_count_list_tf.append(tl_dt_count_tf)
        
        if tl_dt[0] not in tl_dt_sen:
            tl_dt_sen.append(tl_dt[0])
            if tl_dt[0] == 'None':
                tl_dt_count_list_idf.append([0])
            else:
                tl_dt_count_list_idf.append([1])
        else:
            tl_dt_count_list_idf.append([0])
            
    else:
        if tl_dt[0] not in tl_dt_list[temp_index]:
            tl_dt_list[temp_index].append(tl_dt[0]) # temp_index = row index of the dt
            if tl_dt[0] == 'None':
                tl_dt_count_list[temp_index].append(0)
                tl_dt_count_list_tf[temp_index].append(round((0/tl_sen_len), 6))
            else:
                tl_dt_count_list[temp_index].append(1)
                tl_dt_count_list_tf[temp_index].append(round((1/tl_sen_len), 6)) 
            
            if tl_dt[0] not in tl_dt_sen: # tl_dt_sen = list of dts in  a single sentence
                tl_dt_sen.append(tl_dt[0])
                if tl_dt[0] == 'None':
                    tl_dt_count_list_idf[temp_index].append(0)
                else:
                    tl_dt_count_list_idf[temp_index].append(1)
            else:
                tl_dt_count_list_idf[temp_index].append(0)
            
        else: # if the dt is already in tl_dt_list
            temp_dt_index = tl_dt_list[temp_index].index(tl_dt[0])
            if tl_dt[0] == 'None':
                tl_dt_count_list[temp_index][temp_dt_index] += 0
                tl_dt_count_list_tf[temp_index][temp_dt_index] += (0/tl_sen_len)
            else:
                tl_dt_count_list[temp_index][temp_dt_index] += 1
                tl_dt_count_list_tf[temp_index][temp_dt_index] += (1/tl_sen_len)
            tl_dt_count_list_tf[temp_index][temp_dt_index] = round(tl_dt_count_list_tf[temp_index][temp_dt_index], 6)
            
            if tl_dt[0] not in tl_dt_sen:
                tl_dt_sen.append(tl_dt[0])
                if tl_dt[0] == 'None':
                    tl_dt_count_list_idf[temp_index][temp_dt_index] += 0
                else:
                    tl_dt_count_list_idf[temp_index][temp_dt_index] += 1

## Computing the TF-IDF

In [ ]:
import math

def get_idf(tl_count_list):
    tl_idf = []
    for tl_count in tl_count_list:
        temp_quo = tl_doc_len/tl_count
        tl_idf.append(abs(math.log10(temp_quo)))
        
    return tl_idf
# end of get_idf

In [ ]:
def get_idf_il(il_verb_count_list_idf):
    il_idf_list = []
    for il_count_list in il_verb_count_list_idf:
        il_idf = []
        for il_count in il_count_list:
            try:
                temp_quo = il_doc_len/il_count
                il_idf.append(round(abs(math.log10(temp_quo)), 6))
            except:
                temp_quo = 0
                il_idf.append(0)
                
        il_idf_list.append(il_idf)
    return il_idf_list

In [ ]:
def get_tf_il(il_count_sen_list, il_verb_count_list_tf):
    il_tf_list = []
    
    for il_count_list in il_verb_count_list_tf:
        il_tf = []

        for il_count_sen in il_count_sen_list:
            temp_sum = 0
            
            for il_count in il_count_sen:
                temp_sum += il_count
                
            temp_len = len(il_count_sen)
            temp_quo = temp_sum/temp_len
            il_tf.append(round(temp_quo, 6))
        
    return il_tf

In [ ]:
def get_tf(tl_count_sen_list):
    tl_tf = []

    for tl_count_sen in tl_count_sen_list:
        temp_sum = 0
        
        for tl_count in tl_count_sen:
            temp_sum += tl_count
            
        temp_len = len(tl_count_sen)
        temp_quo = temp_sum/temp_len
        tl_tf.append(round(temp_quo, 6))
        
    return tl_tf

In [ ]:
def get_tf_idf(tl_tf, tl_idf):
    tf_idf_tl = []
    
    for i in range(len(tl_tf)):
        temp_score = tl_tf[i] * tl_idf[i]
        tf_idf_tl.append(round(temp_score, 2))
        
    return tf_idf_tl
# end of get_tf_idf

In [ ]:
def get_tf_idf_il(il_tf_list, il_idf_list):
    il_tf_idf_list = []
    
    for i in range(len(il_tf_list)):
        il_tf_idf = []
        for j in range(len(il_tf_list[i])):
            temp_score = il_tf_list[i][j] * il_idf_list[i][j]
            il_tf_idf.append(round(temp_score, 2))
            
        il_tf_idf_list.append(il_tf_idf)
        
    return il_tf_idf_list
# end of get_tf_idf_il

## Tagalog to Ilokano Matcher

In [ ]:
import math

wp_index = None # word position index

"""
instantiating the verb lists
"""

def match_il_tl_pos():
    """
    This function matches the POS of the sentences in the Tagalog and Ilokano datasets
    """
    sp_index = 0
    
    il_sw_list = []
    tl_sw_list = []
    il_verb_list = []
    tl_verb_list = []
    il_noun_list = []
    tl_noun_list = []
    il_adj_list = []
    tl_adj_list = []
    il_adv_list = []
    tl_adv_list = []
    il_conj_list = []
    tl_conj_list = []
    il_prepo_list = []
    tl_prepo_list = []
    il_dt_list = []
    tl_dt_list = []
    il_to_tl_verb_list = []
    """
    instantiating the verb lists
    """
    
    tl_verb_count_list = []
    tl_noun_count_list = []
    tl_adj_count_list = []
    tl_adv_count_list = []
    tl_conj_count_list = []
    tl_prepo_count_list = []
    tl_dt_count_list = []
    
    il_verb_count_list = []
    il_noun_count_list = []
    il_adj_count_list = []
    il_adv_count_list = []
    il_conj_count_list = []
    il_prepo_count_list = []
    il_dt_count_list = []

    il_verb_count_list_tf = []
    il_noun_count_list_tf = []
    il_adj_count_list_tf = []
    il_adv_count_list_tf = []
    il_conj_count_list_tf = []
    il_prepo_count_list_tf = []
    il_dt_count_list_tf = []

    tl_verb_count_list_tf = []
    tl_noun_count_list_tf = []
    tl_adj_count_list_tf = []
    tl_adv_count_list_tf = []
    tl_conj_count_list_tf = []
    tl_prepo_count_list_tf = []
    tl_dt_count_list_tf = []

    il_verb_term_count = []
    il_noun_term_count = []
    il_adj_term_count = []
    il_adv_term_count = []
    il_conj_term_count = []
    il_prepo_term_count = []
    il_dt_term_count = []
    
    il_sen_len_list = []
    tl_sen_len_list = []
    
    tl_verb_count_list_idf = []
    tl_noun_count_list_idf = []
    tl_adj_count_list_idf = []
    tl_adv_count_list_idf = []
    tl_conj_count_list_idf = []
    tl_prepo_count_list_idf = []
    tl_dt_count_list_idf = []
    
    tl_verb_count_list_tf = []
    tl_noun_count_list_tf = []
    tl_adj_count_list_tf = []
    tl_adv_count_list_tf = []
    tl_conj_count_list_tf = []
    tl_prepo_count_list_tf = []
    tl_dt_count_list_tf = []
    
    for il_sen_pos in il_sen_poss_list:
        # loop for getting the pos structure of every sentence
        """
        il_sen is a list of POS of a sentence
        eg. ['VB', 'DT', 'NN', 'DT', 'NN']
        """
        matched = []
        tl_sen = tl_sen_poss_list[sp_index]
        tl_sen_len = len(tl_sen)
        il_sen_len = len(il_sen_pos)
        tl_sen_len = len(tl_sen)
        
        il_sen_len_list.append(il_sen_len)
        tl_sen_len_list.append(tl_sen_len)
        
        wp_index = 0
        """
        instantiating the variables
        """
        
        il_verb_sen = []
        il_noun_sen = []
        il_adj_sen = []
        il_adv_sen = []
        il_conj_sen = []
        il_prepo_sen = []
        il_dt_sen = []
        
        tl_verb_sen = []
        tl_noun_sen = []
        tl_adj_sen = []
        tl_adv_sen = []
        tl_conj_sen = []
        tl_prepo_sen = []
        tl_dt_sen = []
        
        for il_word_pos in il_sen_pos:
            # loop for eail pos in a sentence
            """
            il_word_pos is a POS of a word
            eg. 'VB'
            """
            
            tl_verb = []
            tl_noun = []
            tl_adj = []
            tl_adv = []
            tl_conj = []
            tl_prepo = []
            tl_dt = []
            
            tl_verb_count = []
            tl_noun_count = []
            tl_adj_count = []
            tl_adv_count = []
            tl_conj_count = []
            tl_prepo_count = []
            tl_dt_count = []
            
            tl_verb_count_tf = []
            tl_noun_count_tf = []
            tl_adj_count_tf = []
            tl_adv_count_tf = []
            tl_conj_count_tf = []
            tl_prepo_count_tf = []
            tl_dt_count_tf = []

            il_verb_count = []
            il_noun_count = []
            il_adj_count = []
            il_adv_count = []
            il_conj_count = []
            il_prepo_count = []
            il_dt_count = []
            
            
            il_word = il_pos_data['Tokenized'][sp_index][wp_index]
            # gets the word in every sentence
            
            try:
                curr_tl_pos = tl_sen[wp_index] # ti
            except IndexError:
                curr_tl_pos = 'None'
            try:
                next_tl_pos = tl_sen[wp_index + 1]
            except IndexError:
                next_tl_pos = 'None'
            try:
                next2_tl_pos = tl_sen[wp_index + 2]
            except IndexError:
                next2_tl_pos = 'None'
            try:
                next3_tl_pos = tl_sen[wp_index + 3]
            except IndexError:
                next3_tl_pos = 'None'
            try:
                prev_tl_pos = tl_sen[wp_index - 1]
                if (wp_index - 1) < 0:
                    prev_tl_pos = 'None'
            except IndexError:
                prev_tl_pos = 'None'
            """
            getting the current, next, and previous POS in the sentence
            """
            
            # matching Conditions
            
            # 1. SW
            if il_word_pos == 'SW':
                """
                if SW : SW
                if the Tagalog POS is a SW
                """
                tl_word = tl_pos_data['Tokenized'][sp_index]
                il_sw_list.append(il_word)
                tl_sw_list.append(tl_word)
            
            # 2. VB
            if il_word_pos == 'VB':
                """
                Verb matching
                if the POS is a verb, append the index of the verb to the verb list
                """
                
                append_vb_list(il_word, il_verb_list, tl_verb, tl_verb_list, curr_tl_pos, next_tl_pos, next2_tl_pos, matched, sp_index, wp_index, tl_verb_count_list, tl_verb_count, il_verb_count_list, il_verb_count, il_verb_term_count, il_verb_sen, il_verb_count_list_tf, il_sen_len, tl_verb_sen, tl_verb_count_list_idf, tl_verb_count_list_tf, tl_verb_count_tf, tl_sen_len)
                
            # 3. NN
            if il_word_pos == 'NN':
                """
                Noun matching
                if the POS is a noun, append the index of the noun to the noun list
                """
                append_nn_list(il_word, il_noun_list, tl_noun, tl_noun_list, curr_tl_pos, next_tl_pos, matched, sp_index, wp_index, tl_noun_count_list, tl_noun_count, il_noun_count_list, il_noun_count, il_noun_term_count, il_noun_sen, il_noun_count_list_tf, il_sen_len, tl_noun_sen, tl_noun_count_list_idf, tl_noun_count_list_tf, tl_noun_count_tf, tl_sen_len)

            # 4. JJ
            if il_word_pos == 'JJ':
                """
                Adj matching
                if the POS is a adj, append the index of the adj to the adj list
                """
                append_jj_list(il_word, il_adj_list, tl_adj, tl_adj_list, curr_tl_pos, next_tl_pos, next2_tl_pos, matched, sp_index, wp_index, tl_adj_count_list, tl_adj_count, il_adj_count_list, il_adj_count, il_adj_term_count, il_adj_sen, il_adj_count_list_tf, il_sen_len, tl_adj_sen, tl_adj_count_list_idf, tl_adj_count_list_tf, tl_adj_count_tf, tl_sen_len)
            
            # 5. RB
            if il_word_pos == 'RB':
                """
                Adverb matching
                if the POS is a adverb, append the index of the adverb to the adverb list
                """
                append_rb_list(il_word, il_adv_list, tl_adv, tl_adv_list, curr_tl_pos, next_tl_pos, next2_tl_pos, next3_tl_pos, prev_tl_pos, matched, sp_index, wp_index, tl_adv_count_list, tl_adv_count, il_adv_count_list, il_adv_count, il_adv_term_count, il_adv_sen, il_adv_count_list_tf, il_sen_len, tl_adv_sen, tl_adv_count_list_idf, tl_adv_count_list_tf, tl_adv_count_tf, tl_sen_len)
            
            # 6. CC
            if il_word_pos == 'CC':
                """
                Conjunction matching
                if the POS is a conjunction, append the index of the conjunction to the conjunction list
                """
                append_cc_list(il_word, il_conj_list, tl_conj, tl_conj_list, curr_tl_pos, matched, sp_index, wp_index, tl_conj_count_list, tl_conj_count, il_conj_count_list, il_conj_count, il_conj_term_count, il_conj_sen, il_conj_count_list_tf, il_sen_len, tl_conj_sen, tl_conj_count_list_idf, tl_conj_count_list_tf, tl_conj_count_tf, tl_sen_len)
            
            # 7. PR
            if il_word_pos == 'PR':
                """
                Preposition matching
                if the POS is a preposition, append the index of the conjunction to the conjunction list
                """
                append_pr_list(il_word, il_prepo_list, tl_prepo, tl_prepo_list, curr_tl_pos, next_tl_pos, next2_tl_pos, matched, sp_index, wp_index, tl_prepo_count_list, tl_prepo_count, il_prepo_count_list, il_prepo_count, il_prepo_term_count, il_prepo_sen, il_prepo_count_list_tf, il_sen_len, tl_prepo_sen, tl_prepo_count_list_idf, tl_prepo_count_list_tf, tl_prepo_count_tf, tl_sen_len)
            
            # 8. DT
            if il_word_pos == 'DT':
                """
                Determiner matching
                if the POS is a determiner, append the index of the conjunction to the conjunction list
                """
                append_dt_list(il_word, il_dt_list, tl_dt, tl_dt_list, curr_tl_pos, next_tl_pos, next2_tl_pos, matched, sp_index, wp_index, tl_dt_count_list, tl_dt_count, il_dt_count_list, il_dt_count, il_dt_term_count, il_dt_sen, il_dt_count_list_tf, il_sen_len, tl_dt_sen, tl_dt_count_list_idf, tl_dt_count_list_tf, tl_dt_count_tf, tl_sen_len)
            
                          
            wp_index += 1  
        sp_index += 1
    
    il_verb_idf = get_idf(il_verb_count_list)
    il_noun_idf = get_idf(il_noun_count_list)
    il_adj_idf = get_idf(il_adj_count_list)
    il_adv_idf = get_idf(il_adv_count_list)
    il_conj_idf = get_idf(il_conj_count_list)
    il_prepo_idf = get_idf(il_prepo_count_list)
    il_dt_idf = get_idf(il_dt_count_list)

    il_verb_tf = get_tf(il_verb_count_list_tf)
    il_noun_tf = get_tf(il_noun_count_list_tf)
    il_adj_tf = get_tf(il_adj_count_list_tf)
    il_adv_tf = get_tf(il_adv_count_list_tf)
    il_conj_tf = get_tf(il_conj_count_list_tf)
    il_prepo_tf = get_tf(il_prepo_count_list_tf)
    il_dt_tf = get_tf(il_dt_count_list_tf)
    
    il_verb_tf_idf = get_tf_idf(il_verb_tf, il_verb_idf)
    il_noun_tf_idf = get_tf_idf(il_noun_tf, il_noun_idf)
    il_adj_tf_idf = get_tf_idf(il_adj_tf, il_adj_idf)
    il_adv_tf_idf = get_tf_idf(il_adv_tf, il_adv_idf)
    il_conj_tf_idf = get_tf_idf(il_conj_tf, il_conj_idf)
    il_prepo_tf_idf = get_tf_idf(il_prepo_tf, il_prepo_idf)
    il_dt_tf_idf = get_tf_idf(il_dt_tf, il_dt_idf)
        
    tl_verb_idf = get_idf_il(tl_verb_count_list_idf)
    tl_noun_idf = get_idf_il(tl_noun_count_list_idf)
    tl_adj_idf = get_idf_il(tl_adj_count_list_idf)
    tl_adv_idf = get_idf_il(tl_adv_count_list_idf)
    tl_conj_idf = get_idf_il(tl_conj_count_list_idf)
    tl_prepo_idf = get_idf_il(tl_prepo_count_list_idf)
    tl_dt_idf = get_idf_il(tl_dt_count_list_idf)
    
    tl_verb_tf_idf = get_tf_idf_il(tl_verb_count_list_tf, tl_verb_idf)
    tl_noun_tf_idf = get_tf_idf_il(tl_noun_count_list_tf, tl_noun_idf)
    tl_adj_tf_idf = get_tf_idf_il(tl_adj_count_list_tf, tl_adj_idf)
    tl_adv_tf_idf = get_tf_idf_il(tl_adv_count_list_tf, tl_adv_idf)
    tl_conj_tf_idf = get_tf_idf_il(tl_conj_count_list_tf, tl_conj_idf)
    tl_prepo_tf_idf = get_tf_idf_il(tl_prepo_count_list_tf, tl_prepo_idf)
    tl_dt_tf_idf = get_tf_idf_il(tl_dt_count_list_tf, tl_dt_idf)
    
    dict_il_tl_sw['Ilokano Single Words'] = il_sw_list
    dict_il_tl_sw['Tagalog Single Words'] = tl_sw_list
    
    dict_il_tl_vb['Ilokano Verb'] = il_verb_list
    dict_il_tl_vb['Ilokano Verb TF-IDF'] = il_verb_tf_idf
    dict_il_tl_vb['Ilokano Verb Count'] = il_verb_term_count
    dict_il_tl_vb['Tagalog Verb'] = tl_verb_list
    dict_il_tl_vb['Tagalog Verb Count'] = tl_verb_count_list
    dict_il_tl_vb['Tagalog Verb TF-IDF'] = tl_verb_tf_idf
    
    dict_il_tl_nn['Ilokano Noun'] = il_noun_list
    dict_il_tl_nn['Ilokano Noun TF-IDF'] = il_noun_tf_idf
    dict_il_tl_nn['Ilokano Noun Count'] = il_noun_count_list
    dict_il_tl_nn['Tagalog Noun'] = tl_noun_list
    dict_il_tl_nn['Tagalog Noun Count'] = tl_noun_count_list
    dict_il_tl_nn['Tagalog Noun TF-IDF'] = tl_noun_tf_idf
    
    dict_il_tl_jj['Ilokano Adjective'] = il_adj_list
    dict_il_tl_jj['Ilokano Adjective TF-IDF'] = il_adj_tf_idf
    dict_il_tl_jj['Ilokano Adjective Count'] = il_adj_count_list
    dict_il_tl_jj['Tagalog Adjective'] = tl_adj_list
    dict_il_tl_jj['Tagalog Adjective Count'] = tl_adj_count_list
    dict_il_tl_jj['Tagalog Adjective TF-IDF'] = tl_adj_tf_idf

    dict_il_tl_rb['Ilokano Adverb'] = il_adv_list
    dict_il_tl_rb['Ilokano Adverb TF-IDF'] = il_adv_tf_idf
    dict_il_tl_rb['Ilokano Adverb Count'] = il_adv_count_list
    dict_il_tl_rb['Tagalog Adverb'] = tl_adv_list
    dict_il_tl_rb['Tagalog Adverb Count'] = tl_adv_count_list
    dict_il_tl_rb['Tagalog Adverb TF-IDF'] = tl_adv_tf_idf

    dict_il_tl_cc['Ilokano Conjunction'] = il_conj_list
    dict_il_tl_cc['Ilokano Conjunction TF-IDF'] = il_conj_tf_idf
    dict_il_tl_cc['Ilokano Conjunction Count'] = il_conj_count_list
    dict_il_tl_cc['Tagalog Conjunction'] = tl_conj_list
    dict_il_tl_cc['Tagalog Conjunction Count'] = tl_conj_count_list
    dict_il_tl_cc['Tagalog Conjunction TF-IDF'] = tl_conj_tf_idf

    dict_il_tl_pr['Ilokano Preposition'] = il_prepo_list
    dict_il_tl_pr['Ilokano Preposition TF-IDF'] = il_prepo_tf_idf
    dict_il_tl_pr['Ilokano Preposition Count'] = il_prepo_count_list
    dict_il_tl_pr['Tagalog Preposition'] = tl_prepo_list
    dict_il_tl_pr['Tagalog Preposition Count'] = tl_prepo_count_list
    dict_il_tl_pr['Tagalog Preposition TF-IDF'] = tl_prepo_tf_idf
    
    dict_il_tl_dt['Ilokano Determiner'] = il_dt_list
    dict_il_tl_dt['Ilokano Determiner TF-IDF'] = il_dt_tf_idf
    dict_il_tl_dt['Ilokano Determiner Count'] = il_dt_count_list
    dict_il_tl_dt['Tagalog Determiner'] = tl_dt_list
    dict_il_tl_dt['Tagalog Determiner Count'] = tl_dt_count_list
    dict_il_tl_dt['Tagalog Determiner TF-IDF'] = tl_dt_tf_idf
    
match_il_tl_pos()


In [ ]:
# dict_il_tl_sw.head()
# dict_il_tl_vb.head(50)
# dict_il_tl_nn.head(50)
# dict_il_tl_jj.head(50)
# dict_il_tl_rb.head(50)
# dict_il_tl_cc.head(50)
# dict_il_tl_pr.head(50)
dict_il_tl_dt.head(50)

# temp_dict.head()

## Exporting the dictionary in the json file

### Saving Single Word Dictionary 

In [ ]:
import json

dict_sw = dict_il_tl_sw.to_dict('records')

try:
    with open("../../src/json data/Ilokano to Tagalog/Example-Based/dict_il_sw.json", "w") as outfile:
        json.dump(dict_sw, outfile)
    print("successfully saved the dict_sw.json file")
except:
    print("Error in saving the dict_sw.json file")

### Saving Verb Dictionary 

In [ ]:
dict_vb = dict_il_tl_vb.to_dict('records')

try:
    with open("../../src/json data/Ilokano to Tagalog/Example-Based/dict_il_vb.json", "w") as outfile:
        json.dump(dict_vb, outfile)
    print("successfully saved the dict_vb.json file")
except:
    print("Error in saving the dict_vb.json file")

### Saving Noun Dictionary 

In [ ]:
dict_nn = dict_il_tl_nn.to_dict('records')

try:
    with open("../../src/json data/Ilokano to Tagalog/Example-Based/dict_il_nn.json", "w") as outfile:
        json.dump(dict_nn, outfile)
    print("successfully saved the dict_nn.json file")
except:
    print("Error in saving the dict_nn.json file")

### Saving Adjective Dictionary 

In [ ]:
dict_jj = dict_il_tl_jj.to_dict('records')

try:
    with open("../../src/json data/Ilokano to Tagalog/Example-Based/dict_il_jj.json", "w") as outfile:
        json.dump(dict_jj, outfile)
    print("successfully saved the dict_jj.json file")
except:
    print("Error in saving the dict_jj.json file")

### Saving Adverb Dictionary 

In [ ]:
dict_rb = dict_il_tl_rb.to_dict('records')

try:
    with open("../../src/json data/Ilokano to Tagalog/Example-Based/dict_il_rb.json", "w") as outfile:
        json.dump(dict_rb, outfile)
    print("successfully saved the dict_rb.json file")
except:
    print("Error in saving the dict_rb.json file")

### Saving Conjunction Dictionary 

In [ ]:
dict_cc = dict_il_tl_cc.to_dict('records')

try:
    with open("../../src/json data/Ilokano to Tagalog/Example-Based/dict_il_cc.json", "w") as outfile:
        json.dump(dict_cc, outfile)
    print("successfully saved the dict_cc.json file")
except:
    print("Error in saving the dict_cc.json file")

### Saving Preposition Dictionary 

In [ ]:
dict_pr = dict_il_tl_pr.to_dict('records')

try:
    with open("../../src/json data/Ilokano to Tagalog/Example-Based/dict_il_pr.json", "w") as outfile:
        json.dump(dict_pr, outfile)
    print("successfully saved the dict_pr.json file")
except:
    print("Error in saving the dict_pr.json file")

### Saving Determiner Dictionary 

In [ ]:
dict_dt = dict_il_tl_dt.to_dict('records')

try:
    with open("../../src/json data/Ilokano to Tagalog/Example-Based/dict_il_dt.json", "w") as outfile:
        json.dump(dict_dt, outfile)
    print("successfully saved the dict_dt.json file")
except:
    print("Error in saving the dict_dt.json file")

In [ ]:
temp_sen_list = []

temp_sen_list.append([1])
# temp_sen_list[0].append(1)
temp_sen_list[0][0] += 1
print(temp_sen_list)

In [ ]:
temp_arr = ''

print(len(temp_arr))